- **Alan Delon Sousa Rocha**
- **C-Jovem/Uece**
- **Ciécia de Dados - Intermediário 01/2025**

# Oficina 6: Desenvolvendo um sistema de classificação.

**Vencimento: domingo, 2 fev. 2025, 23:59**

Imagine que você foi contratado por uma empresa de tecnologia para trabalhar em um projeto desafiador: melhorar o sistema de e-mail da empresa. Atualmente, muitos e-mails indesejados (spam) estão passando despercebidos pelo filtro atual, causando transtornos para os funcionários. Sua missão é desenvolver um sistema de classificação mais eficiente que consiga diferenciar com precisão os e-mails “spam” dos “não spam”.

Você possui acesso a um banco de dados de e-mails antigos, já rotulados como “spam” ou “não spam”, e seu chefe pediu para que você proponha uma solução usando aprendizado supervisionado. Com base no que você aprendeu sobre algoritmos de classificação, qual você escolheria para este desafio e por quê? Explique como o algoritmo funciona, por que ele seria eficaz para esse problema específico e qualquer outra consideração relevante que deva ser levada em conta ao implementar a solução. Para ajudá-lo a se guiar na sua resposta, veja alguns aspectos que você pode levar em consideração:

1. **Contextualização**: Explicar o cenário do problema, identificando a
necessidade de uma solução de classificação eficiente.

2. **Escolha do Algoritmo**: Indicar claramente o algoritmo escolhido.

3. **Justificativa Técnica**: Explicar como o algoritmo funciona, detalhando sua aplicabilidade ao problema de classificação de e-mails.

4. **Adequação ao Problema:** Justificar por que o algoritmo é adequado, considerando aspectos como eficiência, facilidade de implementação e possíveis limitações.

5. **Considerações Adicionais:** Incluir qualquer consideração adicional relevante ao contexto do problema, como a natureza dos dados ou a complexidade computacional.




É hora de colocar o seu conhecimento em prática, vamos lá!

# 1.Preparação:

  A empresa está sofrendo com a deficiente classificação de e-mails como spam. isso acarreta em sobecarga da caixa de entrada dos funcionários que perdem comunicados importantes, perdem tempo apagando os emails indesejados e diminui a segurança da comunicação interna. Dessa forma, é necessário identificar os problemas do modelo atual e substitui-lo por outro corrigido e melhorado.

  ## Modelo de Regressão Logística
  -----
  O modelo escolhido para a classificação foi o de regressão logística pois é um modelo ideal para problemas de classificação binária como a de detecção de spam, pois utiliza a funcção sigmoíde que comprime a saída entre 0 e 1. Assim como, é eficiente em lidar com alto número de features (texto vetorizado que usaremos), exige baixo custo computacional é de fácil interpretação.


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression  # Importar o classificador Logistic Regression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt
import kagglehub



path = kagglehub.dataset_download("balaka18/email-spam-classification-dataset-csv")


print("Path to dataset files:", path)
df = pd.read_csv(path + "/emails.csv")

100%|██████████| 1.66M/1.66M [00:00<00:00, 32.7MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/balaka18/email-spam-classification-dataset-csv/versions/1


## 1.2 Pré-processamento

Do banco de dados antigo foi necessário aplicar técnicas a tornar cada mensagem inteligível pelo modelo escolhido, que por ser essencialmente matemaático não consegue lidar com os caracteres.

  - Remoção de caracteres não alfabéticos:
  - Tokenização: Divisão do texto em palavras individuais.
  - Remoção de stopwords e acentuações.: Eliminação de palavras irrelevantes (ex: "e", "de", "o") e acentuações.
  - Normalização: Converção do texto para minúsculas.
  - Vetorização com TF-IDF: cria uma matriz com cada palavra em uma coluna e cada linha a frequência da mesma em cada e-mail.

Com tudo isso as mensagens foram organizadas em uma dataframe com as seguintes características:

   - Linhas: 5172 linhas representando cada email.
   - Colunas:
    - 3002 colunas, sendo a primeira coluna o título do e-mail e a última coluna o rótulo para previsão: 1 para spam, 0 para não spam.
    - As 3000 colunas são as palavras mais comuns em todos os e-mails, após a normalização.

In [ ]:
df.head()

,Email No.,the,to,ect,and,for,of,a,you,hou,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
0,Email 1,0,0,1,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Email 2,8,13,24,6,6,2,102,1,27,...,0,0,0,0,0,0,0,1,0,0
2,Email 3,0,0,1,0,0,0,8,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Email 4,0,5,22,0,5,1,51,2,10,...,0,0,0,0,0,0,0,0,0,0
4,Email 5,7,6,17,1,5,2,57,0,9,...,0,0,0,0,0,0,0,1,0,0


O dataset foi dividido em uma proporção de 80% para teste e 20% para treino. Iso é importante pois garante que o modelo seja testado em dados diferentes do que foram treinados permitindo identificar problemas antes que seja posto em produção.

In [2]:
y = df['Prediction']
X = df.drop(['Prediction', 'Email No.'],  axis=1)


X_train, X_test, y_train, y_test = train_test_split(X,y , test_size = 0.2, random_state = 42)
print(y)

0       0
1       0
2       0
3       0
4       0
       ..
5167    0
5168    0
5169    1
5170    1
5171    0
Name: Prediction, Length: 5172, dtype: int64


# 2. Treinamento

As configurações do modelo de regressão logística foram ajustados com o objetivo de otimizar o treinamento na identificação de spams levando em consideração ser um problema de classificação binária. Os parametros foram acertados com as seguintes características:

  - **penalty='l2'**: L2 (Ridge) é uma técnica para evitar overfitting  ao adicionar uma penalização aos coeficientes do modelo, ideal para dados com features correlacionadas, como nessa classificação de email, pois reduz a magnitude dos coeficientes sem zerá-los como no L1.
  - **solver = 'liblinear':** Define o algoritmo de otimização usado para ajustar os coeficientes do modelo.Foi escolhidoo liblinear pois é adequado para pequenos a médios conjuntos de dados e para classificação binária.
  - **C=0.1** = Controla a força de regularização, 0.1 indica uma regularização forte, útil para evitar overfitting.
  - **max_iter=1000**: Número máximo de iterações para o algoritmo convergir.




In [3]:
model = LogisticRegression(penalty='l2', C=0.1, solver='liblinear', max_iter=1000)
model.fit(X_train,y_train)

LogisticRegression(C=0.1, max_iter=1000, solver='liblinear')

# 3. Avaliação

- **Curva AUC**: Taxa de 0.99%, indicando que o modelo quase nunca se confunde entre as duas classes. Métrica muito importante no cenário encontrado de desbalancemento de classes, em que a classe 1 (é spam) é menos presente que a classe 0 (não é spam), e de classificação binária pois mede a capacidade do modelo em distinguir entre as classes pois pe independente do threshold.

-  **Matriz da confusão:** O modelo obteve baixa quantidade de falsos positivos(15) e falsos negativo (10), destacando a  a boa precisão e sensibilidade do modelo em classificar os e-mails.

- **Relatório de classificação:** Apesar de um desbalanceamento entre as classes o modelo apresenta uma ótima taxa de precisão e sensibilidade;


In [5]:
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:,1]
print("Matriz de Confusão:\n", confusion_matrix(y_test, y_pred))
print("\nRelatório:\n", classification_report(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_proba))


Matriz de Confusão:
 [[724  15]
 [ 10 286]]

Relatório:
               precision    recall  f1-score   support

           0       0.99      0.98      0.98       739
           1       0.95      0.97      0.96       296

    accuracy                           0.98      1035
   macro avg       0.97      0.97      0.97      1035
weighted avg       0.98      0.98      0.98      1035

AUC: 0.9906054566068098


# Conclusão

O modelo treinado comprovou ser o ideal para resolver o problema exposto pois retorna ótimos índices de avaliação. Entretanto, é necessário mais testes em situações de emails com mensagens mais complexas, pois a regressão logística pode não lidar bem em componentes com relações não-lineares.